In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms

import sklearn
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
# seed 고정 (rand 함수 실행시 같은 결과 나올 수 있도록)
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)



# 0. Convolutional Layer



In [ ]:
# Define the convolutional layer
conv = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding='same')

for p in conv.parameters():
    print(p)

In [ ]:
# Define the input tensor
input = torch.randn(10, 3, 7, 7) # Batch_size, Kernel_size, Height, Weight

# Apply the convolution
output = conv(input)

# Print the output shape
print(output.shape)

# 1. CNN

### Dataset Preparation

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("data", train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64, shuffle=True
)


### CNN 선언

In [ ]:
# 모델 정의
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = FashionMNISTModel()

In [ ]:
# Loss, Optimizer 선언
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Loop
model.train()

for epoch in tqdm(range(5)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 1, 28, 28)
        labels = labels

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, 5, i + 1, len(train_loader), loss.item()))

In [ ]:
# GPU 지정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(model.to(device))
next(model.parameters()).device # model parameter 위치 확인 (cuda or cpu)

In [ ]:
# Loss, Optimizer 재 선언
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 모델 학습

In [ ]:
# Training Loop using GPU
for epoch in tqdm(range(5)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 1, 28, 28).to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        # if (i + 1) % 100 == 0:
        #     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
        #         epoch + 1, 5, i + 1, len(train_loader), loss.item()))

### 모델 평가

In [ ]:
model.eval()

y_pred, y_true = torch.zeros([64]).to(device), torch.zeros([64]).to(device)
total, correct = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 1, 28, 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        if (predicted.shape[0]!=64):
            predicted = torch.cat([predicted, torch.zeros(64-predicted.shape[0]).to(device)])
            labels = torch.cat([labels, torch.zeros(64-labels.shape[0]).to(device)])

        y_pred = torch.cat([y_pred, predicted])
        y_true = torch.cat([y_true, labels])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy: {}%'.format((correct / total) * 100))

In [ ]:
print(classification_report(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy()))